In [37]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

In [38]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""
city_code = 'BOG'
    #year-month-day
todays_date = '2022-09-28' #When the offer ends
customer_date_desde = '2022-09-22' #When the offer is available 


tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [39]:
query = """
SELECT DISTINCT
    bo.submit_date,
    TO_CHAR(bo.submit_date, 'YYYY-mm') AS month,
    bo.order_id,
    bo.customer_id,
    boi.name,
    (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio} as gmv_usd,
    COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) as discount_applied,
    COALESCE(foi.total_tax_iva/{tipo_cambio},0) as taxes,
    boi.quantity* foi.step_unit as cant,
    UPPER(fpro.unit) as unidades,
    bs.sku_id,
    bs.addl_product_id as padre_sku_id,
    s.site_identifier_value as region_code,
    CASE
        WHEN COALESCE(bcat2.name, bcat.name) ILIKE 'Proteínas%' THEN 'Proteínas'
        ELSE COALESCE(bcat2.name, bcat.name) 
    END AS category,
    CASE
        WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
        WHEN bcat.name ILIKE 'Res%' THEN 'Res'
        WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE bcat.name
    END AS subcat,
    baid.adjustment_reason
FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                 fow        ON fow.owner_id = ffg.owner_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_product"               fpro       ON fpro.product_id = bp.product_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_customer"              fc         ON fc.customer_id = bo.customer_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_business_type"         bt         ON fc.business_type_id = bt.business_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat2      ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(bo.submit_date) between '{desde}' AND '{hasta}'
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value IN ('{ciudad}')
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   AND fc.business_type_id IN (1)
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
""".format(tipo_cambio=t_cambio, desde=customer_date_desde, hasta=todays_date, ciudad=city_code)

dataframe = read_connection_data_warehouse.runQuery(query)
dataframe[["gmv_usd", "cant"]] = dataframe[["gmv_usd", "cant"]].astype(float)
dataframe["submit_date"] = pd.to_datetime(dataframe["submit_date"])
dataframe["submit_date"] = dataframe["submit_date"].dt.strftime("%d-%m-%y")

In [40]:
dataframe.columns

Index(['submit_date', 'month', 'order_id', 'customer_id', 'name', 'gmv_usd',
       'discount_applied', 'taxes', 'cant', 'unidades', 'sku_id',
       'padre_sku_id', 'region_code', 'category', 'subcat',
       'adjustment_reason'],
      dtype='object')

In [41]:
def get_info(dff):
    df = dff.groupby(by=['subcat']).apply(#the change will be here padre_sku_id // padre_sku_id
        lambda d: pd.Series(
            {
                #"name": d['name'].unique(),
                "gmv_usd": d["gmv_usd"].sum(),
                "cant": d["cant"].sum(),
                "discount_usd": d["discount_applied"].sum(),
                "customer_id": d["customer_id"].nunique(),
                "avg_gmv": d["gmv_usd"].sum() / d["customer_id"].nunique(),
                "avg_cant": d["cant"].sum() / d["customer_id"].nunique(),
                "avg_discount": float(d["discount_applied"].sum() / d["customer_id"].nunique()),
                "% dct": float(d["discount_applied"].sum()) *100.0 / d["gmv_usd"].sum()

            }
        )
    ).reset_index()
    return df

In [42]:
def delta(control_results,test_results,control,piloto):
    delta_df = pd.DataFrame()
    delta_df['subcat'] = test_results['subcat']
    #delta_df['name'] = test_results['name']
    #delta_df['total_control_customers'] = control
    #delta_df['total_test_customers'] = piloto
    delta_df['control_customers'] = control_results['customer_id']
    delta_df['test_customers'] = test_results['customer_id']
    delta_df['delta_avg_gmv'] = test_results['avg_gmv'] - control_results['avg_gmv']
    delta_df['delta_avg_cant'] = test_results['avg_cant'] - control_results['avg_cant']
    delta_df['delta_avg_discount'] = test_results['avg_discount'] - control_results['avg_discount']
    delta_df['delta_conversion'] = ((delta_df['test_customers']/piloto) - (delta_df['control_customers']/control))*100.00
    delta_df['delta_% dct'] = test_results['% dct'] - control_results['% dct']
    return  delta_df

In [43]:
#Colocar CSVs de los experimentos
#SD/export_data/Below_Media/MDE/2022-08-26_csv_MDE_COCACOLA_All_CLIENTS.csv

#exp_sin_restriccion = pd.read_csv(f"SD/export_data/Below_Media/{city_code}/SPO_OFFER_csv_All_SH.csv")#['costumer_id'].unique().tolist()
#clients_exp_sin_restriccion = pd.read_csv(f"SD/export_data/Below_Media/{city_code}/SPO_CLIENTS_csv_All_SH.csv")#['costumer_id'].unique().tolist()


exp_sin_restriccion = pd.read_csv(f"SD/export_data/Below_Media/{city_code}/{customer_date_desde}_OFFER_csv_{city_code}_All_SH.csv")#['costumer_id'].unique().tolist()
clients_exp_sin_restriccion = pd.read_csv(f"SD/export_data/Below_Media/{city_code}/{customer_date_desde}_CLIENTS_csv_{city_code}_All_SH.csv")#['costumer_id'].unique().tolist()

In [44]:
exp_sin_restriccion

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_2022-09-22_BOG_Harinas,NaN,8,True,22-09-2022,26-09-2022,5,2,83766,254112#254113#147339#490566
1,SH_2022-09-22_BOG_Desechables,NaN,7,True,22-09-2022,26-09-2022,10,2,83767,575830#57817#71944#82996#102355
2,SH_2022-09-22_BOG_Carne,NaN,10,True,22-09-2022,26-09-2022,20,2,83768,505439#464554#464556#419101


In [45]:
delta_df = []
control_df = []
test_df = []

for offer in exp_sin_restriccion.values:
    
    control = list(map(int,clients_exp_sin_restriccion['CONTROL_'+offer[0]].dropna().unique()))
    piloto = list(map(int,clients_exp_sin_restriccion['TEST_'+offer[0]].dropna().unique()))
    
    if isinstance(offer[-1], str):
        skus = list(map(int,offer[-1].split("#")))
    else:
        skus = [offer[-1]]
    
    df_test = dataframe[(dataframe.sku_id.isin(skus)) & (dataframe.customer_id.isin(piloto)) & (dataframe.submit_date >= offer[4]) & (dataframe.submit_date <= offer[5])] 
    df_control = dataframe[(dataframe.sku_id.isin(skus)) & (dataframe.customer_id.isin(control)) & (dataframe.submit_date >= offer[4]) & (dataframe.submit_date <= offer[5])]
    
    print(f'Test: {offer[0]} -> control users: {len(control)} / piloto users: {len(piloto)}')
    print(f'Test: {df_test.adjustment_reason.unique()}\n -> control: {df_control.adjustment_reason.unique()}')
    if df_control.empty and df_test.empty:
        print('DataFrame is empty!')
        control_results = pd.DataFrame(columns = ['sku_id', 'name', 'gmv_usd', 'cant', 'customer_id', 'discount_usd',
       'avg_gmv', 'avg_cant', 'avg_discount', '% dct'])
        test_results = pd.DataFrame(columns = ['sku_id', 'name', 'gmv_usd', 'cant', 'customer_id', 'discount_usd',
       'avg_gmv', 'avg_cant', 'avg_discount', '% dct']) 
    elif df_control.empty and (not df_test.empty):     
        print('Control DataFrame is empty!')
        test_results = get_info(df_test)
        control_results = pd.DataFrame(columns = ['sku_id', 'name', 'gmv_usd', 'cant', 'customer_id', 'discount_usd',
       'avg_gmv', 'avg_cant', 'avg_discount', '% dct'])
    elif df_test.empty and (not df_control.empty):
        print('Test DataFrame is empty!')
        control_results = get_info(df_control)
        test_results = pd.DataFrame(columns = ['sku_id', 'name', 'gmv_usd', 'cant', 'customer_id', 'discount_usd',
       'avg_gmv', 'avg_cant', 'avg_discount', '% dct'])   
    else:
        control_results = get_info(df_control)
        test_results = get_info(df_test)
        
    #------------------------#-------------------------#
    control_df.append(control_results)
    test_df.append(test_results)
    delta_df.append(delta(control_results,test_results,len(control),len(piloto)))

Test: SH_2022-09-22_BOG_Harinas -> control users: 139 / piloto users: 787
Test: ['SH_2022-09-22_BOG_Harinas'
 'Hook_BOG_Harina_de_trigo_indupan_panificacion_x_50_kg_Bulto_x_50kg_-_Desde_1_Bulto_4_%_SEP_1_2022'
 'Hook_BOG_Harina_Haz_de_Oros_50kg_Bulto_x_50kg_-_Unidad_4_%_SEP_1_2022']
 -> control: ['Hook_BOG_Harina_de_trigo_indupan_panificacion_x_50_kg_Bulto_x_50kg_-_Desde_1_Bulto_4_%_SEP_1_2022'
 'Hook_BOG_Harina_Haz_de_Oros_50kg_Bulto_x_50kg_-_Unidad_4_%_SEP_1_2022']
Test: SH_2022-09-22_BOG_Desechables -> control users: 415 / piloto users: 2349
Test: ['SH_2022-09-16_BOG_Desechables' 'SH_2022-09-22_BOG_Desechables'
 'Hook_BOG_Contenedor_Sopa_16oz_con_Tapa_Plástica_Darnel_25_Paquetes_x_20_unidades_-_Unidad_2_%_SEP_1_2022'
 None '20220901_BOG_corrientazo_piloto_NB_contenedores_W35']
 -> control: ['Hook_BOG_Contenedor_Sopa_16oz_con_Tapa_Plástica_Darnel_25_Paquetes_x_20_unidades_-_Unidad_2_%_SEP_1_2022'
 'SH_2022-09-16_BOG_Desechables' None]
Test: SH_2022-09-22_BOG_Carne -> control users: 1

In [50]:
test_df[1]

,subcat,gmv_usd,cant,discount_usd,customer_id,avg_gmv,avg_cant,avg_discount,% dct
0,Contenedores,3946.41364,1600.0,308.7702408,344,11.472133,4.651163,0.897588,7.824072


In [51]:
control_df[1]

,subcat,gmv_usd,cant,discount_usd,customer_id,avg_gmv,avg_cant,avg_discount,% dct
0,Contenedores,559.085006,270.0,26.3638160,60,9.318083,4.5,0.439397,4.715529


In [52]:
delta_df[1]

,subcat,control_customers,test_customers,delta_avg_gmv,delta_avg_cant,delta_avg_discount,delta_conversion,delta_% dct
0,Contenedores,60,344,2.154049,0.151163,0.458191,0.186698,3.108543
